# Métrica 2_1_01 — Congruencia entre ingresos por actividad financiera e inversiones declaradas

In [2]:
import traceback
from pymongo import MongoClient, UpdateOne
from pymongo.errors import ConnectionFailure, OperationFailure, BulkWriteError
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- CONFIGURACIÓN ---
METRIC_ID = "2_1_01_CONGRUENCIA_INGRESOS_FINANCIEROS_VS_INVERSIONES"

def get_ingresos_financieros(doc):
    """
    Devuelve el valor numérico de los ingresos por actividad financiera.
    """
    try:
        ingreso = (
            doc.get("declaracion", {})
               .get("situacionPatrimonial", {})
               .get("ingresos", {})
               .get("actividadFinanciera", {})
               .get("remuneracionTotal", {})
               .get("valor", 0)
        )
        return float(ingreso) if ingreso is not None else 0
    except Exception:
        return 0

def tiene_inversiones(doc):
    """
    Devuelve True si existen inversiones declaradas.
    """
    try:
        inversiones = (
            doc.get("declaracion", {})
               .get("situacionPatrimonial", {})
               .get("inversiones", {})
               .get("inversion", [])
        )
        if isinstance(inversiones, list) and len(inversiones) > 0:
            return True
        return False
    except Exception:
        return False

def evaluar_congruencia(doc):
    """
    Aplica la lógica de la métrica 2_1_01 según el Excel.
    """
    ingresos = get_ingresos_financieros(doc)
    inversiones = tiene_inversiones(doc)

    if ingresos == 0:
        return "CUMPLE"  # No tiene ingresos financieros → congruente
    elif ingresos > 0 and inversiones:
        return "CUMPLE"
    elif ingresos > 0 and not inversiones:
        return "NO_CUMPLE"
    else:
        return "SIN_DATO"

def procesar_metrica_2_1_01():
    client = None
    operaciones = []
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0}
    total = 0

    try:
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        source = db[SOURCE_COLLECTION_NAME]
        target = db[METRICS_COLLECTION_NAME]

        print(f"Procesando métrica {METRIC_ID}...")

        cursor = source.find({}, {
            "_id": 1,
            "declaracion.situacionPatrimonial.ingresos.actividadFinanciera.remuneracionTotal": 1,
            "declaracion.situacionPatrimonial.inversiones.inversion": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1
            resultado = evaluar_congruencia(doc)
            resultados[resultado] += 1

            operaciones.append(UpdateOne(
                {"_id": doc["_id"]},
                {"$set": {METRIC_ID: resultado}},
                upsert=True
            ))

            if len(operaciones) == 1000:
                target.bulk_write(operaciones)
                operaciones.clear()
                print(f"  > Procesados {total} documentos...")

        if operaciones:
            target.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Documentos procesados: {total}")
        for k, v in resultados.items():
            print(f"  > {k}: {v}")

    except Exception as e:
        print(f"Error general: {e}")
        traceback.print_exc()
    finally:
        if client:
            client.close()
            print("Conexión cerrada.")

if __name__ == "__main__":
    procesar_metrica_2_1_01()

Procesando métrica 2_1_01_CONGRUENCIA_INGRESOS_FINANCIEROS_VS_INVERSIONES...
  > Procesados 1000 documentos...
  > Procesados 2000 documentos...
  > Procesados 3000 documentos...
  > Procesados 4000 documentos...
  > Procesados 5000 documentos...
  > Procesados 6000 documentos...
  > Procesados 7000 documentos...
  > Procesados 8000 documentos...
  > Procesados 9000 documentos...
  > Procesados 10000 documentos...
  > Procesados 11000 documentos...
  > Procesados 12000 documentos...
  > Procesados 13000 documentos...
  > Procesados 14000 documentos...
  > Procesados 15000 documentos...
  > Procesados 16000 documentos...
  > Procesados 17000 documentos...
  > Procesados 18000 documentos...
  > Procesados 19000 documentos...
  > Procesados 20000 documentos...
  > Procesados 21000 documentos...
  > Procesados 22000 documentos...
  > Procesados 23000 documentos...
  > Procesados 24000 documentos...
  > Procesados 25000 documentos...
  > Procesados 26000 documentos...
  > Procesados 27000 d